In [52]:
include("./prepare_data.jl")
include("./model.jl")

import .Model
import Flux

In [53]:
model = Model.model

Chain(
  Recur(
    LSTMCell(128 => 64),                # 49_536 parameters
  ),
  MaxPool((2,)),
  Recur(
    LSTMCell(32 => 16),                 # 3_168 parameters
  ),
  Flux.flatten,
  Dense(48 => 3),                       # 147 parameters
  NNlib.softmax,
)         # Total: 12 trainable arrays, 52_851 parameters,
          # plus 4 non-trainable, 160 parameters, summarysize 207.309 KiB.

In [54]:
x = rand(Float32, (128,3,4))
model(x)

3×4 Matrix{Float32}:
 0.332245  0.329028  0.328266  0.324593
 0.32893   0.327201  0.326686  0.331657
 0.338826  0.343771  0.345048  0.34375